# Building your first OpenAI Assistant

In [ ]:
from dotenv import load_dotenv
import json
from openai import  AsyncOpenAI
from openai.types.beta import Thread
from openai.types.beta.threads import Run
from pydantic import BaseModel
from typing import Dict, List

In [ ]:
load_dotenv(override=True)

## 1. Define the Assistant class

In [ ]:
class AssistantResult(BaseModel):
    response: str
    thread_id: str

class MaxTurnsReachedException(Exception):
    def __init__(self):
        super().__init__("Reached maximum number of turns")

In [ ]:
class Assistant:
    def __init__(self, assistant_id: str, tools: Dict[str, callable]):
        self.client = AsyncOpenAI()
        self.assistant_id = assistant_id
        self.assistant = None
        self.tools = tools
    
    async def retrieve_assistant(self):
        if self.assistant is None:
            self.assistant = await self.client.beta.assistants.retrieve(self.assistant_id)
        return self.assistant
    
    async def run(self, query: str, thread_id: str = None, max_turns: int = 5) -> AssistantResult:
        await self.retrieve_assistant()

        print(f"Running assistant with thread_id {thread_id}")

        if(thread_id is None):
            thread: Thread = await self.client.beta.threads.create()
            print(f"Created new thread with id {thread.id}")
        else:
            thread: Thread = await self.client.beta.threads.retrieve(thread_id)
            print(f"Retrieved thread with id {thread.id}")
        
        print(f"Sending query to thread {thread.id}: {query}")
        await self.client.beta.threads.messages.create(
            thread_id=thread.id, role="user", content=query
        )

        run: Run = await self.client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
        )

        for turn in range(max_turns):

            # Fetch the last message from the thread
            messages = await self.client.beta.threads.messages.list(
                thread_id=thread.id,
                run_id=run.id,
                order="desc",
                limit=1,
            )
            # print(f"Fetched last message from thread {thread.id}: {messages}")

            # Check for the terminal state of the Run.
            # If state is "completed", exit agent loop and return the LLM response.
            if run.status == "completed":
                print(f"Run completed for thread {thread.id}")
                assistant_res: str = next(
                    (
                        content.text.value
                        for content in messages.data[0].content
                        if content.type == "text"
                    ),
                    None,
                )

                return AssistantResult(thread_id=thread.id, response=assistant_res)
            
            # If state is "requires_action", function calls are required. Execute the functions and send their outputs to the LLM.
            if run.status == "requires_action":
                func_tool_outputs = []

                # LLM can ask for multiple functions to be executed. Execute all function calls in loop and
                # append the results into `func_tool_outputs` list.
                for tool in run.required_action.submit_tool_outputs.tool_calls:
                    # parse the arguments required for the function call from the LLM response
                    args = (
                        json.loads(tool.function.arguments)
                        if tool.function.arguments
                        else {}
                    )

                    try:
                        print("Requiring function call {} with args {} for thread {}".format(tool.function.name, args, thread.id))
                        func_output = await self.tools[tool.function.name](**args)
                        print("Function outputs: {}".format(func_output))
                    except Exception as e:
                        print("Error in running function {}: {}".format(tool.function.name, e))
                        func_output = f'Error in running function {tool.function.name}: {e}'

                    # OpenAI needs the output of the function call against the tool_call_id
                    func_tool_outputs.append(
                        {"tool_call_id": tool.id, "output": str(func_output)}
                    )

                # Submit the function call outputs back to OpenAI
                run = await self.client.beta.threads.runs.submit_tool_outputs_and_poll(
                    thread_id=thread.id, run_id=run.id, tool_outputs=func_tool_outputs
                )

                # Continue the agent loop.
                # Agent will check the output of the function output submission as part of next iteration.
                continue

            # Handle errors if terminal state is "failed"
            else:
                if run.status == "failed":
                    print(
                        f"OpenAIFunctionAgent turn-{turn+1} | Run failure reason: {run.last_error}"
                    )

                raise Exception(
                    f"Failed to generate text due to: {run.last_error}"
                )
        
        # Raise error if turn-limit is reached.
        await self.client.beta.threads.runs.cancel(run.id,thread_id=thread_id)
        raise MaxTurnsReachedException()
    
    async def cancel_thread_run(self, thread_id: str):
        thread: Thread = await self.client.beta.threads.retrieve(thread_id)
        run: Run = await self.client.beta.threads.runs.list(thread_id=thread.id).data[0]

        await self.client.beta.threads.runs.cancel(run.id,thread_id=thread_id)

## 2. Define the Assistant Tools

In [ ]:
async def calculate(what: str) -> float:
    """
    e.g. calculate: 4 * 7 / 3
    Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.
    """
    return eval(what)


async def average_price_per_weight(name: str) -> str:
    """
    e.g. average_price_per_weight: mortadella
    returns average price per weight of the provided cold cut (e.g. mortadella, prosciutto cotto, etc.)
    """
    name = name.lower()
    if name == "bresaola":
        return "Bresaola has an average price of about 40 euros per kilogram."
    elif name == "prosciutto crudo" or name == "prosciutto":
        return "Prosciutto crudo (cured ham) has an average price of about 30 euros per kilogram."
    elif name == "prosciutto cotto":
        return "Prosciutto cotto (cooked ham) has an average price of about 15 euros per kilogram."
    elif name == "mortadella":
        return "Mortadella has an average price of about 12 euros per kilogram."
    elif name == "pancetta":
        return "Pancetta has an average price of about 18 euros per kilogram."
    else:
        return "Unknown product"

In [ ]:
#
# Function schemas - copy and paste into OpenAI Playground
#

# {
#   "name": "calculate",
#   "description": "Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.",
#   "strict": true,
#   "parameters": {
#     "type": "object",
#     "required": [
#       "what"
#     ],
#     "properties": {
#       "what": {
#         "type": "string",
#         "description": "A string expression representing the calculation to perform (e.g. '4 * 7 / 3')"
#       }
#     },
#     "additionalProperties": false
#   }
# }

# {
#   "name": "average_price_per_weight",
#   "description": "Returns average price per weight of the provided cold cut (e.g. mortadella, prosciutto cotto, etc.)",
#   "strict": true,
#   "parameters": {
#     "type": "object",
#     "required": [
#       "name"
#     ],
#     "properties": {
#       "name": {
#         "type": "string",
#         "description": "The name of the cold cut for which the average price per weight is requested"
#       }
#     },
#     "additionalProperties": false
#   }
# }

## 3. Create the OpenAI Assistant using the OpenAI Playground

1. Navigate to [OpenAI Assistant Playground](https://platform.openai.com/playground/assistants)
2. Create a new Assistant and name following the format: `<name_first_letter>_<surname>_first_agent`
3. Define the System Instructions
4. Select gpt-4o-mini as the model
5. Generate a new Function definition

## 4. Instantiate the Assistant

In [ ]:
#
# System prompt - copy and paste into OpenAI Playground
#

# # Task
# You are SalumiereGPT, an helpful assistant for a supermarket. Your goal is to calculate the price of the requested food by the user.
# You have several tools available that help you to get the prices and calculate the final cost. 

# # Output
# - Provide the price detail if multiple items are requested
# - Keep your answer concise

# # Example session:
# Question: How much is 1kg of mortadella?
# Thought: I should get the mortadella price using average_price_per_weight
# Action: average_price_per_weight: mortadella

# (You will be called again with this:)

# Observation: Mortadella has an average price of about 12 euros per kilogram.

# (You then output:)

# 1kg of mortadella is 12 euro

In [ ]:
# ASSISTANT_ID = "<YOUR_ASSISTANT_ID>"
ASSISTANT_ID = "asst_dbf562R2LG7PaYwzSDTgnqW0"

TOOLS = {
    "calculate": calculate,
    "average_price_per_weight": average_price_per_weight
    }

In [ ]:
# Create the assistant instance
assistant = Assistant(ASSISTANT_ID, TOOLS)

## 5. Use the Assistant

In [34]:
await assistant.run(query="Hi, who are you?", max_turns=5)

Running assistant with thread_id None
Created new thread with id thread_ZUyTXDX5rkla5uWB1ztqwK80
Sending query to thread thread_ZUyTXDX5rkla5uWB1ztqwK80: Hi, who are you?
Run completed for thread thread_ZUyTXDX5rkla5uWB1ztqwK80


AssistantResult(response='I am SalumiereGPT, your helpful assistant for supermarket queries, specifically related to food prices. How can I assist you today?', thread_id='thread_ZUyTXDX5rkla5uWB1ztqwK80')

In [22]:
question = "How much are 76g of mortadella?"

res = await assistant.run(query=question, max_turns=5)
print("###########################################\n")
print(f"Thread ID: {res.thread_id}\n")
print(res.response)

Running assistant with thread_id None
Created new thread with id thread_es51baxcxQHe1yeU0Vqb3YGD
Sending query to thread thread_es51baxcxQHe1yeU0Vqb3YGD: How much are 76g of mortadella?
Requiring function call average_price_per_weight with args {'name': 'mortadella'} for thread thread_es51baxcxQHe1yeU0Vqb3YGD
Function outputs: Mortadella has an average price of about 12 euros per kilogram.
Requiring function call calculate with args {'what': '12 * 76 / 1000'} for thread thread_es51baxcxQHe1yeU0Vqb3YGD
Function outputs: 0.912
Run completed for thread thread_es51baxcxQHe1yeU0Vqb3YGD
###########################################

Thread ID: thread_es51baxcxQHe1yeU0Vqb3YGD

76g of mortadella is approximately 0.91 euros.


In [23]:
question = "I need 200g of pancetta, 150g of prosciutto crudo, and 45g of bresaola. How much is it?"

res = await assistant.run(query=question, max_turns=5)
print("###########################################\n")
print(f"Thread ID: {res.thread_id}\n")
print(res.response)

Running assistant with thread_id None
Created new thread with id thread_0Wh0IzXPF8ij8ZXIKvrHnVHP
Sending query to thread thread_0Wh0IzXPF8ij8ZXIKvrHnVHP: I need 200g of pancetta, 150g of prosciutto crudo, and 45g of bresaola. How much is it?
Requiring function call average_price_per_weight with args {'name': 'pancetta'} for thread thread_0Wh0IzXPF8ij8ZXIKvrHnVHP
Function outputs: Pancetta has an average price of about 18 euros per kilogram.
Requiring function call average_price_per_weight with args {'name': 'prosciutto crudo'} for thread thread_0Wh0IzXPF8ij8ZXIKvrHnVHP
Function outputs: Prosciutto crudo (cured ham) has an average price of about 30 euros per kilogram.
Requiring function call average_price_per_weight with args {'name': 'bresaola'} for thread thread_0Wh0IzXPF8ij8ZXIKvrHnVHP
Function outputs: Bresaola has an average price of about 40 euros per kilogram.
Requiring function call calculate with args {'what': '0.2 * 18'} for thread thread_0Wh0IzXPF8ij8ZXIKvrHnVHP
Function outp